<a href="https://colab.research.google.com/github/Moh2106/youtube_spam_classifier/blob/main/youtube_spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import library
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string

### Chargement du dataset

In [ ]:
# Chargement de mon dataset
youtube_psy = pd.read_csv("/home/youtube-spam-collection/Youtube01-Psy.csv")
youtube_katyperry = pd.read_csv("/home/youtube-spam-collection/Youtube02-KatyPerry.csv")
youtube_lmfao = pd.read_csv("/home/youtube-spam-collection/Youtube03-LMFAO.csv")
youtube_eminem = pd.read_csv("/home/youtube-spam-collection/Youtube04-Eminem.csv")
youtube_shakira = pd.read_csv("/home/youtube-spam-collection/Youtube05-Shakira.csv")

dataset = pd.concat([youtube_psy, youtube_katyperry, youtube_lmfao, youtube_eminem, youtube_shakira], axis=0)

In [ ]:
dataset

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
365,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
366,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
367,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
368,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0


In [ ]:
# Voir si mon dataset est équilibré
class_count = dataset['CLASS'].value_counts()
class_count

1    1005
0     951
Name: CLASS, dtype: int64

### Prétraitement des content de youtube

In [ ]:
content = dataset['CONTENT']
content = content.to_list

In [ ]:
# Téléchargement des stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
english = set(stopwords.words('english'))
print(english)

{'with', "couldn't", 'll', "hasn't", 'a', 'if', 've', 'some', 'above', 'own', 'which', 'between', 'theirs', 'your', 'of', 'in', 'few', 'or', 'needn', 'until', 'ours', 'up', 'while', 'shan', "isn't", 'then', "mightn't", 'that', 'my', "she's", 'not', 'very', 'mightn', 'd', "aren't", "you'd", 'both', 'hadn', 'any', 'each', 'as', 'than', "haven't", 'weren', 'what', 'have', 'won', "you'll", 'y', 'yourselves', 'most', 'here', "wouldn't", 'don', 'shouldn', 'should', 'having', 'were', 'mustn', "mustn't", 're', 'is', 'does', 'wouldn', "that'll", 'our', 'me', 'against', 'yourself', 'only', "didn't", 'further', 'was', 'but', 'about', 'out', 'once', 'himself', 'can', 'from', 'to', 'there', 'hers', 'these', 'ma', 'by', 'we', 'o', 'has', 'an', 'more', 'them', 'are', 'she', "don't", 'itself', 'during', 'didn', 'had', 'for', 'him', 's', 'couldn', 'ain', "weren't", 'their', 'nor', 'they', 'myself', "doesn't", 'who', 'before', 'ourselves', "it's", "you're", 'it', 'on', 'over', 'other', 'so', 'same', 'no

In [ ]:
# Remove Stopword
def removePunctuation(sentence):
  result = [word.lower() for word in sentence if word not in string.punctuation]
  return "".join(result)

In [ ]:
youtube_dataframe = pd.DataFrame()
youtube_dataframe['class'] = dataset['CLASS']

In [ ]:
youtube_dataframe

,class
0,1
1,1
2,1
3,1
4,1
...,...
365,0
366,0
367,0
368,0


In [ ]:
youtube_dataframe['corpus_without_stopword'] = dataset['CONTENT'].apply(removePunctuation)
youtube_dataframe

,class,corpus_without_stopword
0,1,huh anyway check out this youtube channel koby...
1,1,hey guys check out my new channel and our firs...
2,1,just for test i have to say murdevcom
3,1,me shaking my sexy ass on my channel enjoy ﻿
4,1,watchvvtarggvgtwq check this out ﻿
...,...,...
365,0,i love this song because we sing it at camp al...
366,0,i love this song for two reasons 1it is about ...
367,0,wow
368,0,shakira u are so wiredo


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Ecrire une fonction pour tokinezer le corpus de colonne « t_s_p ».
def tokenizeTexte(sentence):
  sentence = sentence.lower()
  return word_tokenize(sentence, language="english")

In [ ]:
youtube_dataframe['corpus_without_tokenization'] = youtube_dataframe['corpus_without_stopword'].apply(tokenizeTexte)
youtube_dataframe

,class,corpus_without_stopword,corpus_without_tokenization
0,1,huh anyway check out this youtube channel koby...,"[huh, anyway, check, out, this, youtube, chann..."
1,1,hey guys check out my new channel and our firs...,"[hey, guys, check, out, my, new, channel, and,..."
2,1,just for test i have to say murdevcom,"[just, for, test, i, have, to, say, murdevcom]"
3,1,me shaking my sexy ass on my channel enjoy ﻿,"[me, shaking, my, sexy, ass, on, my, channel, ..."
4,1,watchvvtarggvgtwq check this out ﻿,"[watchvvtarggvgtwq, check, this, out, ﻿]"
...,...,...,...
365,0,i love this song because we sing it at camp al...,"[i, love, this, song, because, we, sing, it, a..."
366,0,i love this song for two reasons 1it is about ...,"[i, love, this, song, for, two, reasons, 1it, ..."
367,0,wow,[wow]
368,0,shakira u are so wiredo,"[shakira, u, are, so, wiredo]"


In [ ]:
stemming = PorterStemmer()
def applyStemming(texte):
  result = [stemming.stem(word) for word in texte]
  return " ".join(result)

In [ ]:
youtube_dataframe['corpus_stemming'] = youtube_dataframe['corpus_without_tokenization'].apply(applyStemming)
youtube_dataframe

,class,corpus_without_stopword,corpus_without_tokenization,corpus_stemming
0,1,huh anyway check out this youtube channel koby...,"[huh, anyway, check, out, this, youtube, chann...",huh anyway check out thi youtub channel kobyos...
1,1,hey guys check out my new channel and our firs...,"[hey, guys, check, out, my, new, channel, and,...",hey guy check out my new channel and our first...
2,1,just for test i have to say murdevcom,"[just, for, test, i, have, to, say, murdevcom]",just for test i have to say murdevcom
3,1,me shaking my sexy ass on my channel enjoy ﻿,"[me, shaking, my, sexy, ass, on, my, channel, ...",me shake my sexi ass on my channel enjoy ﻿
4,1,watchvvtarggvgtwq check this out ﻿,"[watchvvtarggvgtwq, check, this, out, ﻿]",watchvvtarggvgtwq check thi out ﻿
...,...,...,...,...
365,0,i love this song because we sing it at camp al...,"[i, love, this, song, because, we, sing, it, a...",i love thi song becaus we sing it at camp all ...
366,0,i love this song for two reasons 1it is about ...,"[i, love, this, song, for, two, reasons, 1it, ...",i love thi song for two reason 1it is about af...
367,0,wow,[wow],wow
368,0,shakira u are so wiredo,"[shakira, u, are, so, wiredo]",shakira u are so wiredo


### Separate features and target

In [ ]:
features = youtube_dataframe['corpus_stemming']
target = youtube_dataframe['class']

In [ ]:
# Utilisation de la méthode CountVectorizer pour réprésenter numériquement notre dataset
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(features)

In [ ]:
# Afficher le vocabulaire
print("*** Vocabulaire ***")
print(len(vectorizer.get_feature_names_out()))

print("\n Matrice de fréquences")
print(X.toarray())

*** Vocabulaire ***
3840

 Matrice de fréquences
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), target, test_size = 0.20, random_state=42)

In [ ]:
print(f"Shape de X_train {X_train.shape}")
print(f"Shape de X_est {X_test.shape}")

Shape de X_train (1564, 3840)
Shape de X_est (392, 3840)


### Entrainons notre modèle avec le RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Créer un modèle random forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrainer le modèle
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Prédiction du modèle
y_pred = model.predict(X_test)

In [ ]:
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0,

In [ ]:
# Evaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Afficher les résultats
print(f"Précision : {accuracy}")
print("Rapport de classification :\n", report)

Précision : 0.8877551020408163
Rapport de classification :
               precision    recall  f1-score   support

           0       0.82      0.97      0.89       176
           1       0.97      0.82      0.89       216

    accuracy                           0.89       392
   macro avg       0.89      0.89      0.89       392
weighted avg       0.90      0.89      0.89       392

